In [7]:
# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()
# DOcplex
from docplex.mp.model import Model
# Aqua
from qiskit import BasicAer
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.optimization.ising import docplex, max_cut, tsp
from qiskit.optimization.ising.common import sample_most_likely
# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

Credentials are already in use. The existing account in the session will be replaced.


In [8]:
# Create an instance of a model and variables
mdl = Model(name='max_vars')
x = {i: mdl.binary_var(name='x_{0}'.format(i)) for i in range(1,5)}

# Objective function
max_vars_func = mdl.sum(x[i] for i in range(1,5))
mdl.maximize(max_vars_func)

# Constraints
mdl.add_constraint(mdl.sum(i*x[i] for i in range(1,5)) == 3)

print(mdl.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: max_vars

Maximize
 obj: x_1 + x_2 + x_3 + x_4
Subject To
 c1: x_1 + 2 x_2 + 3 x_3 + 4 x_4 = 3

Bounds
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1

Binaries
 x_1 x_2 x_3 x_4
End



In [9]:
qubitOp, offset = docplex.get_operator(mdl)

In [10]:
# checking the full Hamiltonian
ee = ExactEigensolver(qubitOp, k=1)
result = ee.run()

print('energy:', result['energy'])
print('objective:', result['energy'] + offset)

x = sample_most_likely(result['eigvecs'][0])
print('solution:', x)

energy: -57.5
objective: -2.0
solution: [1. 1. 0. 0.]


In [6]:
# Statevector simulation
seed = 10598

spsa = SPSA(max_trials=300)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa)

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('solution objective:', result['energy'] + offset)
print('solution:', x)

energy: -56.31828013436072
time: 18.990363121032715
solution objective: -0.8182801343607196
solution: [0. 0. 1. 0.]


In [11]:
# run quantum algorithm with shots
seed = 10598

spsa = SPSA(max_trials=300)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('solution objective:', result['energy'] + offset)
print('solution:', x)

energy: -55.0771484375
time: 37.28726840019226
solution objective: 0.4228515625
solution: [0 0 1 0]


In [12]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright